In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
avp = pd.read_excel("../data/raw/AVPdb_data.xls")

In [5]:
mavp = pd.read_excel("../data/raw/mAVPdb_data.xls")

In [11]:
avp_data = avp[["Id", "Sequence", "Family", "Inhibition/IC50", "Unit"]]

In [15]:
mavp_data = mavp[["ID", "SEQUENCE", "FAMILY", "Inhibition/IC50", "UNIT"]]
mavp_data.columns = avp_data.columns

In [87]:
avp_data.shape

(1817, 5)

In [60]:
def get_positive_amp_data(path):
    return pd.read_csv(path, sep=",", header=0).reset_index().drop('index', axis=1)[['Sequence', 'Activity']]

OTHER_ALPHABETS = "UOXBZJ"

def contains(other_alphabets, seq):
    for o in str(other_alphabets):
        if o in str(seq):
            return True
    return False

def trim_all(strings):
    return list(set(value.strip().strip(',').lower() for value in strings))

def sequence_filtering(data):
    sequences = data[data.apply(lambda r: not contains(OTHER_ALPHABETS, r['Sequence']), axis=1)]
    sequences = sequences[sequences.apply(lambda r: not str(r['Sequence']) == 'nan', axis=1)]
    sequences['Sequence'] = sequences['Sequence'].apply(lambda x: x.upper())
    return sequences

def massage_camp_data(data):
    sequences = sequence_filtering(data)
    sequences['Activity'] = sequences['Activity'].apply(lambda x: trim_all(str(x).split(',')))
    # sequences['Activity'] = sequences['Activity'].apply(lambda x: x.remove(''))
    sequences = sequences.reset_index().drop('index', axis=1)
    for i in range(sequences.__len__()):
        while '' in sequences.loc[i, 'Activity']:
            sequences.loc[i, 'Activity'].remove('')
    return sequences

In [75]:
avp_data = sequence_filtering(avp_data)
avp_data = avp_data.drop_duplicates(["Sequence"]).reset_index(drop=True)

In [86]:
avp_data.shape

(1817, 5)

In [61]:
tw_curated = get_positive_amp_data("../data/raw/positive_data_filtered2807.csv")
tw_curated = massage_camp_data(tw_curated).reset_index(drop=True)
tw_curated = tw_curated.drop_duplicates(["Sequence"]).reset_index(drop=True)

In [85]:
tw_curated.shape

(2729, 2)

In [63]:
mlb = preprocessing.MultiLabelBinarizer()
tw_curated_target = mlb.fit_transform(tw_curated.Activity)
tw_curated_target = pd.DataFrame(tw_curated_target, columns=mlb.classes_)

In [64]:
tw_curated_target

,antibacterial,anticancer,antifungal,antimicrobial,antiparasitic,antiviral,nan
0,1,0,0,1,0,0,0
1,1,0,0,1,0,0,0
2,1,0,1,1,0,0,0
3,1,0,1,1,0,0,0
4,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...
2724,0,0,0,0,0,1,0
2725,1,0,0,0,0,0,0
2726,1,0,0,0,0,0,0
2727,1,0,0,1,0,0,0


In [65]:
tw_curated_avp = tw_curated[tw_curated_target['antiviral'] == 1].reset_index(drop=True)

In [84]:
tw_curated_avp.shape

(194, 2)

In [58]:
tw_curated_avp.to_csv("../data/raw/tw_curated_avp.csv", index=False)

In [77]:
avp_sequences = pd.concat([tw_curated_avp.Sequence, avp_data.Sequence], axis=0).reset_index(drop=True)

In [82]:
avp_sequences.shape

(2004,)

In [80]:
avp_sequences = avp_sequences.drop_duplicates().reset_index(drop=True)

In [83]:
avp_sequences.shape

(2004,)

In [88]:
avp_sequences.to_csv("../data/raw/avp_sequences.csv", index=False)

In [5]:
avppred_data_training1 = pd.read_excel("../data/raw/AVPpred_data.xlsx", sheet_name="Training1-AVP")
avppred_data_training1 = avppred_data_training1[["Sequence"]]

In [9]:
avppred_data_validation1 = pd.read_excel("../data/raw/AVPpred_data.xlsx", sheet_name="Validation1-AVP")
avppred_data_validation1 = avppred_data_validation1[["Sequence"]]

In [15]:
avppred_data = pd.concat([avppred_data_training1, avppred_data_validation1])
avppred_data = avppred_data.drop_duplicates(["Sequence"]).reset_index(drop=True)

In [16]:
avp_seq = pd.read_csv("../data/raw/avp_sequences.csv")

In [18]:
avp_seq_complete = pd.concat([avp_seq, avppred_data])
avp_seq_complete = avp_seq_complete.drop_duplicates(["Sequence"]).reset_index(drop=True)

In [20]:
avp_seq_complete.shape

(2314, 1)

In [21]:
avp_seq_complete.to_csv("../data/raw/avp_sequences_tw_avpdb_avppred.csv", index=False)